# PineconeStore

[Pinecone](https://www.pinecone.io/) 是一个向量数据库，为全球一些顶尖公司的 AI 提供支持。

本指南提供了关于如何开始使用 Pinecone [向量存储](/docs/concepts/#vectorstores) 的快速概述。如需所有 `PineconeStore` 功能和配置的详细文档，请访问 [API 参考](https://api.js.langchain.com/classes/langchain_pinecone.PineconeStore.html)。

## 概述

### 集成详情

| 类 | 包 | [Python 支持](https://python.langchain.com/docs/integrations/vectorstores/pinecone/) | 包的最新版本 |
| :--- | :--- | :---: | :---: |
| [`PineconeStore`](https://api.js.langchain.com/classes/langchain_pinecone.PineconeStore.html) | [`@langchain/pinecone`](https://npmjs.com/@langchain/pinecone) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/pinecone?style=flat-square&label=%20&) |

## 设置

要使用 Pinecone 向量存储，你需要创建一个 Pinecone 账户、初始化一个索引并安装 `@langchain/pinecone` 集成包。你还需安装 [官方 Pinecone SDK](https://www.npmjs.com/package/@pinecone-database/pinecone) 以初始化一个客户端，并将其传递给 `PineconeStore` 实例。

本指南还将使用 [OpenAI 嵌入](/docs/integrations/text_embedding/openai)，这需要你安装 `@langchain/openai` 集成包。如果需要，你也可以使用 [其他支持的嵌入模型](/docs/integrations/text_embedding)。

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/pinecone @langchain/openai @langchain/core @pinecone-database/pinecone@5
</Npm2Yarn>
```

### 凭据

注册一个 [Pinecone](https://www.pinecone.io/) 账户并创建一个索引。确保其维度与你打算使用的嵌入向量的维度一致（对于 OpenAI 的 `text-embedding-3-small`，默认是 1536）。完成此操作后，请设置 `PINECONE_INDEX`、`PINECONE_API_KEY` 和（可选）`PINECONE_ENVIRONMENT` 环境变量：

```typescript
process.env.PINECONE_API_KEY = "your-pinecone-api-key";
process.env.PINECONE_INDEX = "your-pinecone-index";

// 可选
process.env.PINECONE_ENVIRONMENT = "your-pinecone-environment";
```

如果你在本指南中使用 OpenAI 嵌入，则还需要设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果你想对模型调用进行自动追踪，也可以取消注释以下内容并设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

In [1]:
import { PineconeStore } from "@langchain/pinecone";
import { OpenAIEmbeddings } from "@langchain/openai";

import { Pinecone as PineconeClient } from "@pinecone-database/pinecone";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const pinecone = new PineconeClient();
// Will automatically read the PINECONE_API_KEY and PINECONE_ENVIRONMENT env vars
const pineconeIndex = pinecone.Index(process.env.PINECONE_INDEX!);

const vectorStore = await PineconeStore.fromExistingIndex(
  embeddings,
  {
    pineconeIndex,
    // Maximum number of batch requests to allow at once. Each batch is 1000 vectors.
    maxConcurrency: 5,
    // You can pass a namespace here too
    // namespace: "foo",
  }
);

## 管理向量存储

### 向向量存储中添加项目

In [2]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents, { ids: ["1", "2", "3", "4"] });

[ '1', '2', '3', '4' ]


**注意：** 添加文档后，它们变为可查询之前会有轻微延迟。

### 从向量存储中删除项目

In [3]:
await vectorStore.delete({ ids: ["4"] });

## 查询向量存储

一旦创建了向量存储并添加了相关文档，在链或代理运行期间很可能需要对其进行查询。

### 直接查询

执行一个简单的相似性搜索可以按照以下方式：

In [8]:
// Optional filter
const filter = { source: "https://example.com" };

const similaritySearchResults = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


如果你想执行相似性搜索并获得相应的分数，可以运行：

In [5]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.165] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=0.148] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### 通过转换为检索器进行查询

你还可以将向量存储转换为[检索器](/docs/concepts/retrievers)，以便在你的链中更方便地使用。 

In [6]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});

await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [如何操作：使用RAG进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## API 参考

如需详细了解所有 `PineconeStore` 功能和配置，请访问 [API 参考](https://api.js.langchain.com/classes/langchain_pinecone.PineconeStore.html)。